Now we calculate the q-value $Q$ where 
\begin{equation}
\begin{split}
Q = \Delta mc^2\\
\end{split}
\end{equation}
where $c$ is the speed of light, and $\Delta m$ is the change in mass of the all the nuclei in a reaction. 
For these reactions, the only reactants are the target and the projectile. 






In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using PlotlyJS, CSV, DataFrames
using Printf
using SymPy
using PDFIO
using Unzip
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

WebIO._IJuliaInit()

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

In [2]:
function is_number(input) 
    if isa(input, Number)
        return true
    end
    return tryparse(Float64, input) != nothing
end

is_number (generic function with 1 method)

Now to import and clean the data

In [3]:
amu_path = parent_dir * "ImportedData\\Atomic_mass_table_2020.csv"
masses = CSV.read(amu_path, DataFrame, stringtype=String)[:,2:end]
good_rows = [row for row in 1:size(masses)[1] if is_number(masses[row, end-1])]
masses = masses[good_rows, :]
masses[!,1:3] = Int.(masses[:,1:3])
masses[:,[3,4,end-1]]

,A,Elt.,Atomic mass (u)
,Int64,String,Float64
1,1,n,1.00866
2,1,H,1.00783
3,2,H,2.0141
4,3,H,3.01605
5,3,He,3.01603
6,3,Li,3.03078
7,4,H,4.02643
8,4,He,4.0026
9,4,Li,4.02719


This function returns the precise mass in daltons of a given isotope. Isotopes must be given in the form mass number + symbol in a single string, such as "3H" for tritium or "1n" for a free neutron. 

In [4]:
function precise_mass(input)
    index = 1
    while (index < length(input) && 
            tryparse(Int, string(input[1:index])) != nothing)
        index += 1
    end
    index -= 1
    mass_number = parse(Int, input[1:index])
    symbol = input[index+1:end]
    row = 1
    while ((masses[row,3] != mass_number || masses[row,4] != symbol) 
        && row < size(masses)[1])
        row += 1
    end
    if row == size(masses)[1]
        if string(masses[row,3]) * string(masses[row,4]) == input
            return masses[row, end-1]
        else 
            return "Isotope not in dataset"
        end
    end
    return masses[row, end-1] 
end
precise_mass("3H")

3.01604928132

In [5]:
N_A = 6.02214076*(BigFloat(10)^23)

6.02214076000000009258883437723852694034576416015625e+23

The function q_val returns the q value of a reaction given the products and reactants in eV/reaction. 

In [6]:
kg_per_dalton = 1.66053906660 * BigFloat(10)^(-27)
c = 299792458
joule_per_eV = 1.602176634 * BigFloat(10)^-19
conversion_constant = kg_per_dalton * c^2 / joule_per_eV
total_mass(ingredients) = sum([precise_mass(ingredient) for ingredient in ingredients])
q_val(reactants, products) = (total_mass(reactants) - total_mass(products)) * conversion_constant 
q_val(["3H", "2H"], ["4He","1n"])

1.758929986491761040539044986485839498059167757678315342281902419445550953510919e+07

This agrees with the usually cited q-values for different reactions. 

In [7]:
q_val(["2H", "2H"], ["3He", "1n"])

3.268908850974102367854075994107107301986568434178828839895100956710973926549135e+06

In [8]:
q_val(["2H", "2H"], ["3H", "1H"])

4.032663828738721139338876004089564883296793627391702351816031599817901716929761e+06

In [9]:
q_val(["1n", "9Be"],["8Be", "1n", "1n"])

-1.664538882129621392511337106953757283363676677527588068640109772397040296181692e+06

Now I need to find the reactants and products for a given projectile, MT, and target. Once I do that, I can jus tuse the q_val function :) 

In [35]:
split("3,2", ",")

2-element Vector{SubString{String}}:
 "3"
 "2"

In [59]:
mt_dir = parent_dir * "ExportedData\\MT given projectile\\"
projectiles = ["Alpha", "Deuteron", "Helium3", "Proton", "Neutron", "Tritium"]
projectile_MT_dict = Dict([])
for projectile in projectiles
    df = CSV.read(mt_dir * projectile * "MTS.csv", 
            DataFrame, stringtype=String)
    new_df = DataFrame()
    new_df[!, "MTs"] = [parse.(Int, split(df[row,2][2:end-1], ",")) 
                    for row in 1:size(df)[1]]
    new_df[!, "Reaction"] = [parse.(Int, split(df[row,1][2:end-1], ",")) 
                            for row in 1:size(df)[1]]
    projectile_MT_dict[projectile] = new_df
end
projectile_MT_dict

Dict{Any, Any} with 6 entries:
  "Deuteron" => 14×2 DataFrame…
  "Proton"   => 15×2 DataFrame…
  "Tritium"  => 13×2 DataFrame…
  "Neutron"  => 14×2 DataFrame…
  "Helium3"  => 16×2 DataFrame…
  "Alpha"    => 18×2 DataFrame…

I think I need to programmatically redo the interpretation of MTs to account for multiple sets of nuclei and free particles having the same reaction. 

In each dataframe of interpretted MTs, the first column describes the reaction in terms ($\Delta Z$, $\Delta A$) where $Z$ is the proton number and $A$ the mass number. The second column is a list of MTs that each denote this reaction. 

In [60]:
print(projectile_MT_dict["Proton"])

15×2 DataFrame
 Row │ MTs                                Reaction 
     │ Array…                             Array…   
─────┼─────────────────────────────────────────────
   1 │ [5, 10, 33]                        [0, -3]
   2 │ [207, 801, 849, 108, 116, 109, 1…  [-1, -3]
   3 │ [35, 113]                          [-2, -6]
   4 │ [700, 32, 215, 205, 11, 702, 749…  [0, -2]
   5 │ [115, 1]                           [-1, -2]
   6 │ [101]                              [-2, -2]
   7 │ [699, 650, 104, 651, 28, 41, 204…  [0, -1]
   8 │ [875, 90, 4, 91, 218, 52, 34, 18…  [1, 0]
   9 │ [603, 111, 203, 103, 20, 217, 60…  [0, 0]
  10 │ [114, 117, 45]                     [-2, -5]
  11 │ [36]                               [-2, -7]
  12 │ [30, 25, 29, 24, 22]               [-1, -4]
  13 │ [210, 457, 460, 456, 451, 216, 2…  [1, 1]
  14 │ [251, 253, 151, 751, 750, 27, 45…  [0, 1]
  15 │ [112, 23]                          [-2, -4]

In [63]:
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];
function reaction(projectile, MT)
    df = projectile_MT_dict[projectile]
    row = 1
    while (row < size(df)[1] && !(MT in df[row, "MTs"]))
        row += 1
    end
    if row == size(df)[1]
        if !(MT in df[row, "MTs"])
            return "MT not in dataset"
        end
    end
    return df[row, "Reaction"]
end
reaction("Proton", 1)

2-element Vector{Int64}:
 -1
 -2

In [67]:
function MTs(projectile, reaction)
    df = projectile_MT_dict[projectile]
    row = 1
    while (row < size(df)[1] && reaction != df[row, "Reaction"])
        row += 1
    end
    if row == size(df)[1]
        if !(reaction == df[row, "Reaction"])
            return "Reaction not in dataset"
        end
    end
    return df[row, "MTs"]
end
MTs("Proton", [0,0])

13-element Vector{Int64}:
 603
 111
 203
 103
  20
 217
 600
  21
  38
 649
 601
 604
 602

In [ ]:
get_ingredients(projectile, MT)
    